# Grover's serach for RNA design

In [200]:
import numpy as np
from math import pi,log2,floor,ceil
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

## 1 - Arithmetics

Adder mod 2^n.

In [201]:
#MAJ
q=QuantumRegister(3)
MAJ = QuantumCircuit(q)

MAJ.cnot(q[2],q[1])
MAJ.cnot(q[2],q[0])
MAJ.ccx(q[0],q[1],q[2])

maj = MAJ.to_gate(label='MAJ')

#UMA
q=QuantumRegister(3)
UMA = QuantumCircuit(q)

UMA.ccx(q[0],q[1],q[2])
UMA.cnot(q[2],q[0])
UMA.cnot(q[0],q[1])

uma = UMA.to_gate(label='UMA')

#add
def add_circuit(nb_bits):
    concerved = QuantumRegister(nb_bits, name="=")
    not_concerved = QuantumRegister(nb_bits, name="->")
    initial_carry = QuantumRegister(1)
    add = QuantumCircuit(initial_carry,concerved,not_concerved)
    add.append(maj,[initial_carry,not_concerved[0],concerved[0]])
    for i in range(1,nb_bits):
        add.append(maj,[concerved[i-1],not_concerved[i],concerved[i]])
    for i in range(nb_bits-1,0,-1):
        add.append(uma,[concerved[i-1],not_concerved[i],concerved[i]])
    add.append(uma,[initial_carry,not_concerved[0],concerved[0]])
    return add

def add_gate(nb_bits):
    return add_circuit(nb_bits).to_gate(label="add{}".format(nb_bits))

add_circuit(6).draw()

┌──────┐                                                        »
q100680: ┤0     ├────────────────────────────────────────────────────────»
         │      │┌──────┐                                                »
    =_0: ┤2     ├┤0     ├────────────────────────────────────────────────»
         │      ││      │┌──────┐                                        »
    =_1: ┤      ├┤2     ├┤0     ├────────────────────────────────────────»
         │      ││      ││      │┌──────┐                                »
    =_2: ┤      ├┤      ├┤2     ├┤0     ├────────────────────────────────»
         │  MAJ ││      ││      ││      │┌──────┐                ┌──────┐»
    =_3: ┤      ├┤      ├┤      ├┤2     ├┤0     ├────────────────┤0     ├»
         │      ││  MAJ ││      ││      ││      │┌──────┐┌──────┐│      │»
    =_4: ┤      ├┤      ├┤      ├┤      ├┤2     ├┤0     ├┤0     ├┤2     ├»
         │      ││      ││  MAJ ││      ││      ││      ││      ││      │»
    =_5: ┤      ├┤      ├┤      ├┤      ├┤      ├┤2     ├┤2     ├┤      ├»
         │      ││      ││      ││  MAJ ││      ││      ││      ││      │»
   ->_0: ┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
         └──────┘│      ││      ││      ││  MAJ ││      ││      ││  UMA │»
   ->_1: ────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                 └──────┘│      ││      ││      ││  MAJ ││  UMA ││      │»
   ->_2: ────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                         └──────┘│      ││      ││      ││      ││      │»
   ->_3: ────────────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├»
                                 └──────┘│      ││      ││      ││      │»
   ->_4: ────────────────────────────────┤1     ├┤      ├┤      ├┤1     ├»
                                         └──────┘│      ││      │└──────┘»
   ->_5: ────────────────────────────────────────┤1     ├┤1     ├────────»
                                                 └──────┘└──────┘        »
«                                 ┌──────┐
«q100680: ────────────────────────┤0     ├
«                         ┌──────┐│      │
«    =_0: ────────────────┤0     ├┤2     ├
«                 ┌──────┐│      ││      │
«    =_1: ────────┤0     ├┤2     ├┤      ├
«         ┌──────┐│      ││      ││      │
«    =_2: ┤0     ├┤2     ├┤      ├┤      ├
«         │      ││      ││      ││  UMA │
«    =_3: ┤2     ├┤      ├┤      ├┤      ├
«         │      ││      ││  UMA ││      │
«    =_4: ┤      ├┤      ├┤      ├┤      ├
«         │      ││  UMA ││      ││      │
«    =_5: ┤      ├┤      ├┤      ├┤      ├
«         │  UMA ││      ││      ││      │
«   ->_0: ┤      ├┤      ├┤      ├┤1     ├
«         │      ││      ││      │└──────┘
«   ->_1: ┤      ├┤      ├┤1     ├────────
«         │      ││      │└──────┘        
«   ->_2: ┤      ├┤1     ├────────────────
«         │      │└──────┘                
«   ->_3: ┤1     ├────────────────────────
«         └──────┘                        
«   ->_4: ────────────────────────────────
«                                         
«   ->_5: ────────────────────────────────
«

Increment and decrement.

In [202]:
def incr_circuit(nb_bits):
    number = QuantumRegister(nb_bits,name="number")
    incr = QuantumCircuit(number)
    for i in range(nb_bits-1):
        incr.append(MCXGate(nb_bits-i-1),[number[j] for j in range(nb_bits-i)])
    incr.x(number[0])
    return incr

def incr_gate(nb_bits):
    return incr_circuit(nb_bits).to_gate(label=f"incr{nb_bits}")

def decr_circuit(nb_bits):
    return incr_circuit(nb_bits).inverse()

def decr_gate(nb_bits):
    return decr_circuit(nb_bits).to_gate(label=f"decr{nb_bits}")

incr_circuit(6).draw()
decr_circuit(6).draw()

┌───┐                         
number_0: ┤ X ├──■────■────■────■────■──
          └───┘┌─┴─┐  │    │    │    │  
number_1: ─────┤ X ├──■────■────■────■──
               └───┘┌─┴─┐  │    │    │  
number_2: ──────────┤ X ├──■────■────■──
                    └───┘┌─┴─┐  │    │  
number_3: ───────────────┤ X ├──■────■──
                         └───┘┌─┴─┐  │  
number_4: ────────────────────┤ X ├──■──
                              └───┘┌─┴─┐
number_5: ─────────────────────────┤ X ├
                                   └───┘

In [204]:
#set a register (initially at 0) to a constant
def set_cst_circuit(constant,nb_bits):
    number = QuantumRegister(nb_bits,name="number")
    add_cst = QuantumCircuit(number)
    
    #TODO
    
    return add_cst

#add a constant to a register
def add_cst_circuit(constant,nb_bits):
    number = QuantumRegister(nb_bits,name="number")
    add_cst = QuantumCircuit(number)
    
    #TODO
    
    return add_cst

def add_cst_gate(constant,nb_bits):
    return add_cst_circuit(constant,nb_bits).to_gate(label=f"(+){constant}")

def substract_cst_circuit(constant,nb_bits):
    return add_cst_circuit(-1*constant,nb_bits).inverse()

def substract_cst_gate(constant,nb_bits):
    return substract_cst_circuit(constant,nb_bits).to_gate(label=f"(-){constant}")

## 2 - Framework

Conventions, notations, parameter managment, verify input is well formed, elaborate on raw input (create `length`, `sequence_specification` and `loop_specification` from `folding_specification`).

In [263]:
#keywords to find important comments:
#OPTIMISATION: there is (or may be) an optimisation here
#FUTURE: something that should be done if a certain feature was added in the future
#AMPLITUDE: diffrent configurations does not have the same amplitude, so it may be important for the Grover search
#GOOD PRACTICE: this is NOT a reliable way to do, please change it
#TODO

nb_bits_bracket_dot = 2
nb_bits_base = 2
nb_bits_loop_type = 2
nb_bits_energy = 5 #TODO
min_length_hairpin_loop = 3

def nb_bits_wanna_count_from_zero_until(n):
    return ceil(log2(float(n)))+1

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))[:nb_bits]

def base_to_ctrl_state(base):
    if "A" == base:
        return "00"
    if "C" == base:
        return "01"
    if "G" == base:
        return "10"
    if "U" == base:
        return "11"
    raise Exception("Not a base.")


def bracket_dot_to_ctrl_state(bd):
    if "(" == bd:
        return "01"
    if ")" == bd:
        return "11"
    if "." == bd:
        return "0"
    if "|" == bd:
        return "1"
    if "<-." == bd:
        return "00"
    if ".->" == bd:
        return "10"


def base_to_set(base):
    if "N" == base or "." == base:
        return set(("A","C","G","U"))
    elif "A" == base:
        return set(("A"))
    elif "C" == base:
        return set(("C"))
    elif "G" == base:
        return set(("G"))
    elif "U" == base:
        return set(("U"))
    elif "R" == base:
        return set(("A","G"))
    elif "Y" == base:
        return set(("C","U"))
    elif "S" == base:
        return set(("C","G"))
    elif "W" == base:
        return set(("A","U"))
    elif "K" == base:
        return set(("G","U"))
    elif "M" == base:
        return set(("A","C"))
    elif "B" == base:
        return set(("C","G","U"))
    elif "D" == base:
        return set(("A","G","U"))
    elif "H" == base:
        return set(("A","C","U"))
    elif "V" == base:
        return set(("A","C","G"))
    
    
def base_matching_Watson_Crick(base):
    if "A" == base:
        return "U"
    if "C" == base:
        return "G"
    if "G" == base:
        return "C"
    if "U" == base:
        return "A"
    raise Exception("Not a base.")

def base_pair_complement(base_set):
    complement = set(())
    for base in base_set:
        complement.add(base_matching_Watson_Crick(base))
    return complement

def is_valid_base_pair(left,right):
    left = base_to_set(left)
    right = base_to_set(right)
    right = base_pair_complement(right)
    return not left.intersection(right).issubset(set(()))

def characterise_loop_specification(length,folding_specification,sequence_specification,folding_specification_param,sequence_specification_param):
    sequence_specification_param = (sequence_specification if None == sequence_specification_param else sequence_specification_param)
    
    length = len(folding_specification)
    bottom = [None,None,None,-1,0]
    bottom[0] = bottom
    bottom[1] = bottom
    bottom[2] = bottom
    loop_specification = [[bottom,bottom,bottom,k,0] for k in range(length)]
    
    #semantics of loop_specification
    
    #[0] matching
    #if folding_specification: k_left(-k_right)
    #then, compute loop_specification[k_left][0]=loop_specification[k_right]
    #otherwise let loop_specification[k_left][0]=None
    
    
    #if folding_specification: k_prev(-k_next(
    #or folding_specification: k_prev)-k_next)
    
    #[1] next
    #then, compute loop_specification[k_prev][1]=loop_specification[k_next]
    #otherwise let loop_specification[k_prev][1]=None
    
    #[2] previous
    #then, compute loop_specification[k_next][2]=loop_specification[k_prev]
    #otherwise let loop_specification[k_next][2]=None
    
    #[3] position
    
    #[4] loop length if "(" == folding_specification[k]
    #0 otherwise
    
    open_loop_stack = []
    prev_open = None
    prev_close = None
    for k in range(length):
        if "(" == folding_specification[k]:
            if None != prev_open:
                prev_open[1] = loop_specification[k]
                loop_specification[k][2] = prev_open
            open_loop_stack.append(loop_specification[k])
            prev_open = loop_specification[k]
            prev_close = None
        elif ")" == folding_specification[k]:
            if None != prev_close:
                prev_close[1] = loop_specification[k]
                loop_specification[k][2] = prev_close
            try:
                left = open_loop_stack.pop()
            except IndexError:
                raise Exception(f"Wrong folding specification format: \"{folding_specification_param[k]}\" in folding_specification at index {k} has no matching \"(\".")
            left[0] = loop_specification[k]
            if not is_valid_base_pair(sequence_specification[left[3]],sequence_specification[k]):
                raise Exception(f"Folding specification and sequence specification do not match: "+
                                f"\"{folding_specification_param[left[3]]}\" in folding_specification at index {left[3]} "+
                                f"matches "+
                                f"\"{folding_specification_param[k      ]}\" in folding_specification at index {k}, "+
                                f"but "+
                                f"\"{sequence_specification_param[left[3]]}\" in sequence_specification at index {left[3]} "+
                                f"does not form a valid base pair with "+
                                f"\"{sequence_specification_param[k      ]}\" in sequence_specification at index {k}.")
            prev_close = loop_specification[k]
            prev_open = None
        elif "." == folding_specification[k]:
            if 0 < len(open_loop_stack):
                open_loop_stack[-1][4]+=1
        else:
            raise Exception(f"Wrong folding specification format: \"{folding_specification_param[k]}\" found in folding_specification at index {k}.")
    if 0 < len(open_loop_stack):
        raise Exception(f"Wrong folding specification format: \"{folding_specification_param[open_loop_stack[-1][3]]}\" in folding_specification at index {k} has no matching \")\".")
    return loop_specification

def will_use_registers(circuit,registers):
    for reg in registers:
        if list != type(reg):
            if not circuit.has_register(reg):
                circuit.add_register(reg)
        else:
            for r in reg:
                if not circuit.has_register(r):
                    circuit.add_register(r)


#GOOD PRACTICE: please use classes instead of this!

def to_circuit_builder(append):
    def circuit_builder(folding_specification,sequence_specification=None,*other_args):
        #save parameters for error messages
        folding_specification_param = folding_specification
        sequence_specification_param = sequence_specification
        
        #semantics of sequence_specification
        #follow the IUPAC code
        #R == A or G
        #Y == C or U
        #S == G or C
        #W == A or U
        #K == G or U
        #M == A or C
        #B == C or G or U
        #D == A or G or U
        #H == A or C or U
        #V == A or C or G
        #N == A or C or G or U
        
        #check the input is well formed
        
        folding_specification = str(folding_specification)
        length = len(folding_specification)
        
        if length < min_length_hairpin_loop+2:
            raise Exception(f"folding_specification's length is to small (minimum size is {min_length_hairpin_loop+2}).")
        
        if None == sequence_specification:
            sequence_specification="N"*length
        else:
            sequence_specification=str(sequence_specification).upper()
            if length != len(sequence_specification):
                raise Exception(f"Wrong sequence specification format: sequence_specification's length ({len(sequence_specification_param)}) is different from folding_specification's length ({len(folding_specification_param)}).")
        
        loop_specification = characterise_loop_specification(length,folding_specification,sequence_specification,folding_specification_param,sequence_specification_param)
        
        #TODO: check that characterise_loop_specification() works well
  #      for a in loop_specification:
  #          print(a)
        
        all_specifiers = [length,sequence_specification,folding_specification,loop_specification]
        
        nb_bits_position = nb_bits_wanna_count_from_zero_until(length) #from 0 until length because we add a ( at the begining to check the folding is well formed
        max_enclosed_loop = (length-2)//(2+min_length_hairpin_loop)
        nb_bits_enclosed = 0 if max_enclosed_loop == 0 else nb_bits_wanna_count_from_zero_until(max_enclosed_loop) #from 0 until the maximum number of loops (of folding (...)) that could be enclosed in a multiloop
        #TODO set nb_bits_incorrect
        #nb_bits_incorrect = nb_bits_wanna_count_from_zero_until(length+length)#from 0 because 0==incorrect_configuration when the configuration is correct, length (and not length+1 indeed) is the maximum number of incrementations of incorrect_configuration due to incorrect folding and length is the maximum contribution of incorrect base pairing to incorrect_configuration
        nb_bits_incorrect = 6 #enough for what can be tested on a personnal computer
        
        all_nb_bits=[nb_bits_position,nb_bits_enclosed,nb_bits_incorrect]
        
        incr_position=incr_gate(nb_bits_position)
        decr_position=decr_gate(nb_bits_position)
        add_position=add_gate(nb_bits_position)
        incr_incorrect=incr_gate(nb_bits_incorrect)
        decr_incorrect=decr_gate(nb_bits_incorrect)
        incr_nb_enclosed= (IGate() if 0 == nb_bits_enclosed else incr_gate(nb_bits_enclosed))
        incr_energy=incr_gate(nb_bits_energy)
        decr_energy=decr_gate(nb_bits_energy)
        add_energy=add_gate(nb_bits_energy)
        
        all_gates=[incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy]
        
        #input
        #semantics of folding
        #<-. == "00"
        #.-> == "10"
        #) == "11"
        #( == "01"
        #so
        #. == "x0"
        #| == "x1"
        folding = [QuantumRegister(nb_bits_bracket_dot,name=f"(/./h/){k}") for k in range(length)]
        #semantics of sequence
        #A == "00"
        #C == "01"
        #G == "10"
        #U == "11"
        sequence = [QuantumRegister(nb_bits_base,name=f"A/U/G/C{k}") for k in range(length)]

        #auxillary qbits
        count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed
        continue_searching_right_matching_left = QuantumRegister(1,name="continue_match")
        nb_enclosed = QuantumRegister(nb_bits_enclosed,name="enclosed") #count how many stems are enclosed in the loop
        loop_length = QuantumRegister(nb_bits_position,name="length") #loop_length is initialised to 1 (actually not 0 because of an optimisation used there <**>) and is incremented each time 1 == count and . == folding until 0 == count
        loop_length_right = QuantumRegister(nb_bits_position,name="length_right")
        #in the order of the sequence:
        next_left_prev = QuantumRegister(nb_bits_base,name="next_left_prev")
        next_left = QuantumRegister(nb_bits_base,name="next_left")
        right_matching_next_left_next = QuantumRegister(nb_bits_base,name="next_right_next")
        right_matching_left_prev = QuantumRegister(nb_bits_base,name="right_prev")
        right_matching_left_next = QuantumRegister(nb_bits_base,name="right_next")
        use_right_matching_left_prev = QuantumRegister(1,name="right_prev?")
        use_right_matching_left_next = QuantumRegister(1,name="right_next?")
        
        #results
        incorrect_configuration = QuantumRegister(nb_bits_incorrect,name="incorrect")
        energy = QuantumRegister(nb_bits_energy,name="energy_delta")
        
        all_registers=[folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy]
        
        circuit = QuantumCircuit()

        def semantics(bit_string):
            i_bit = 0
            semantics_string = ""

            def semantics_number(label,nb_bits_number):
                s=str(int(bit_string[i_bit:i_bit+nb_bits_number],2))
                delta=nb_bits_number-len(s)
                return s+"_"+label+delta*" "+" ",nb_bits_number
            
            def semantics_base(label):
                s=("A" if bit_string[i_bit:i_bit+nb_bits_base]==base_to_ctrl_state("A") else
                   "C" if bit_string[i_bit:i_bit+nb_bits_base]==base_to_ctrl_state("C") else
                   "G" if bit_string[i_bit:i_bit+nb_bits_base]==base_to_ctrl_state("G") else
                   "U" if bit_string[i_bit:i_bit+nb_bits_base]==base_to_ctrl_state("U") else "")
                return s+"_"+label+" ",nb_bits_base

            if circuit.has_register(energy):
                s,i=semantics_number("E",nb_bits_energy)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(incorrect_configuration):
                s,i=semantics_number("w",nb_bits_incorrect)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(use_right_matching_left_next):
                s,i=semantics_number("Mn?",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(use_right_matching_left_prev):
                s,i=semantics_number("Mp?",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(right_matching_left_next):
                s,i=semantics_base("Mn")
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(right_matching_left_prev):
                s,i=semantics_base("Mp")
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(right_matching_next_left_next):
                s,i=semantics_base("Rn")
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(next_left):
                s,i=semantics_base("L")
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(next_left_prev):
                s,i=semantics_base("Lp")
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(loop_length_right):
                s,i=semantics_number("n2",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(loop_length):
                s,i=semantics_number("n",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(nb_enclosed):
                s,i=semantics_number("e",nb_bits_enclosed)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(continue_searching_right_matching_left):
                s,i=semantics_number("s",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(count):
                s,i=semantics_number("c",nb_bits_position)
                semantics_string+=s
                i_bit+=i
                
            if circuit.has_register(sequence[0]):
                semantics_string+="\n"
                for k in range(length-1,-1,-1):
                    semantics_string+=("A" if bit_string[i_bit+k*nb_bits_base:i_bit+(k+1)*nb_bits_base]==base_to_ctrl_state("A") else
                                       "C" if bit_string[i_bit+k*nb_bits_base:i_bit+(k+1)*nb_bits_base]==base_to_ctrl_state("C") else
                                       "G" if bit_string[i_bit+k*nb_bits_base:i_bit+(k+1)*nb_bits_base]==base_to_ctrl_state("G") else
                                       "U" if bit_string[i_bit+k*nb_bits_base:i_bit+(k+1)*nb_bits_base]==base_to_ctrl_state("U") else "")
                i_bit+=length*nb_bits_base
            if circuit.has_register(folding[0]):
                semantics_string+="\n"
                for k in range(length-1,-1,-1):
                    semantics_string+=("<" if bit_string[i_bit+k*nb_bits_bracket_dot:i_bit+(k+1)*nb_bits_bracket_dot]==bracket_dot_to_ctrl_state("<-.") else
                                       ">" if bit_string[i_bit+k*nb_bits_bracket_dot:i_bit+(k+1)*nb_bits_bracket_dot]==bracket_dot_to_ctrl_state(".->") else
                                       "(" if bit_string[i_bit+k*nb_bits_bracket_dot:i_bit+(k+1)*nb_bits_bracket_dot]==bracket_dot_to_ctrl_state("(")   else
                                       ")" if bit_string[i_bit+k*nb_bits_bracket_dot:i_bit+(k+1)*nb_bits_bracket_dot]==bracket_dot_to_ctrl_state(")")   else "")
                i_bit+=length*nb_bits_bracket_dot
            return semantics_string
        
        basic_args_to_call_append=[circuit]+all_registers+[all_registers]+all_nb_bits+[all_nb_bits]+all_gates+[all_gates]+all_specifiers+[all_specifiers]
        basic_args_to_call_append.append(basic_args_to_call_append)
        
        label=append(*basic_args_to_call_append,*other_args)
        return circuit,semantics,label
    return circuit_builder

def to_gate_builder(append):
    def gate_builder(length,*other_args):
        circ,_,lab=to_circuit_builder(append)(length,sequence_specification,folding_specification,
                                              *other_args)
        return circ.to_gate(label=lab)
    return gate_builder

def to_gate_inverse_builder(append):
    def gate_inverse_builder(length,sequence_specification,folding_specification,
                             *other_args):
        circ,_,lab=to_circuit_builder(append)(length,sequence_specification,folding_specification,
                                              *other_args)
        return circ.inverse().to_gate(label=lab+"^-1"),
    return gate_inverse_builder

def inversed(append):
    def append_inversed(circuit,
                        folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                        all_registers,
                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                        all_nb_bits,
                        incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                        all_gates,
                        length,sequence_specification,folding_specification,loop_specification,
                        all_specifiers,
                        basic_args_to_call_append,
                        *other_args):
        circ = QuantumCircuit()
        #add QuantumRegisters to the circ and append gates
        basic_args_to_call_append[0]=circ
        lab=append(*basic_args_to_call_append,*other_args)
        basic_args_to_call_append[0]=circuit
        all_registers_list = folding+sequence+[all_registers[i] for i in range(2,len(all_registers))]
        register_list =list(np.asarray(all_registers_list,dtype=Register)[[circ.has_register(reg) for reg in all_registers_list]])
        
        will_use_registers(circuit,register_list)
        qbit_list = [qbit for register in register_list for qbit in register]
        circuit.compose(circ.inverse(),qbit_list,inplace=True)
        return lab+"^-1"
    return append_inversed


## 3 - Groover oracle

### 3.1 - Circuit(s)

The framework enables to get a subcircuit from any function for tests.

Computation of `energy`.

In [264]:
#TODO: extract energy parameters directly from the viennaRNA package for modularity and maintenance
#instead of (simingly 2011-old) tables from the NNDB (Nearest Neighbour model Data Base)
#here https://rna.urmc.rochester.edu/NNDB/turner04/index.html


#TODO: energy parameter conventions

AU_end_penalty = 0.45
helix_flat_contribution = 4.09
special_C_bulge_bonus = -0.9
hairpin_mismatch_UU_AG_contribution = -0.9
hairpin_mismatch_GG_contribution = -0.8
hairpin_C3_contribution = 1.5

#multiloop coefficients
multiloop_flat_contribution = 9.25# ± 0.91
one_branch_contribution = -0.63# ± 0.24
strain_contribution = 3.14# ± 0.44

In [265]:
def mismatch_contribution(base_mismatch5,base_pair5,base_mismatch3):
    return [
        #"A" == base_left
        [[-1.0, -0.8, -1.1, -0.8],
         [-0.7, -0.6, -0.7, -0.5],
         [-1.1, -0.8, -1.2, -0.8],
         [-0.7, -0.6, -0.7, -0.5]],
        
        #"C" == base_left
        [[-1.1, -1.5, -1.3, -1.5],
         [-1.1, -0.7, -1.1, -0.5],
         [-1.6, -1.5, -1.4, -1.5],
         [-1.1, -1.0, -1.1, -0.7]],
        
        #"G" == base_left
        [[-1.5, -1.5, -1.4, -1.5],
         [-1.0, -1.1, -1.0, -0.8],
         [-1.4, -1.5, -1.6, -1.5],
         [-1.0, -1.4, -1.0, -1.2]],
        
        #"U" == base_left
        [[-0.8, -1.0, -0.8, -1.0],
         [-0.6, -0.7, -0.6, -0.7],
         [-0.8, -1.0, -0.8, -1.0],
         [-0.6, -0.8, -0.6, -0.8]]
    ][int(base_to_ctrl_state(base_mismatch5),2)][int(base_to_ctrl_state(base_pair5),2)][int(base_to_ctrl_state(base_mismatch3),2)]


def substract_enclosed_mismatch_contribution(circuit,
                                             folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                                             all_registers,
                                             nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                                             all_nb_bits,
                                             incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                                             all_gates,
                                             length,sequence_specification,folding_specification,loop_specification,
                                             all_specifiers,
                                             basic_args_to_call_append,
                                             k_left,
                                             base_next_left_prev,base_next_left,base_right_matching_next_left_next):
    #parameters base_ are in the order of the sequence    
    will_use_registers(circuit,[folding,sequence,continue_searching_right_matching_left,right_matching_left_next,use_right_matching_left_next,energy])
    should_not_cancel = continue_searching_right_matching_left
    
    #1 is the only value for which an enclosed mismatch is ambiguous:
    #0   -> bulge has no enclosed mismatch
    #>=2 -> . can only go with one | since 1 == nb_enclosed
    
    contribution = mismatch_contribution(base_next_left_prev,base_next_left,base_right_matching_next_left_next)
    
    circuit.append(substract_cst_gate(contribution,nb_bits_energy).
                   control(
                       1+3*nb_bits_base,ctrl_state=
                       base_to_ctrl_state(base_next_left_prev)+
                       base_to_ctrl_state(base_next_left)+
                       base_to_ctrl_state(base_right_matching_next_left_next)+
                       "0"
                   ),
                   [should_not_cancel]+
                   [next_left_prev[i] for i in range(nb_bits_base)]+
                   [next_left[i] for i in range(nb_bits_base)]+
                   [right_matching_next_left_next[i] for i in range(nb_bits_base)]+
                   
                   [energy[i] for i in range(nb_bits_energy)]
                  )
    return "cancel enclosed mismatch"


def add_outside_mismatch_contribution(circuit,
                                      folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                                      all_registers,
                                      nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                                      all_nb_bits,
                                      incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                                      all_gates,
                                      length,sequence_specification,folding_specification,loop_specification,
                                      all_specifiers,
                                      basic_args_to_call_append,
                                      k_left,
                                      base_left_prev,base_left,base_right_matching_left_next):
    #parameters base_ are in the order of the sequence
    
    will_use_registers(circuit,[folding,sequence,continue_searching_right_matching_left,right_matching_left_next,use_right_matching_left_next,energy])
    
    is_not_outside_mismatch = continue_searching_right_matching_left
    contribution = mismatch_contribution(base_left_prev,base_left,base_right_matching_left_next)
    
    circuit.append(add_cst_gate(contribution,nb_bits_energy).
                   control(
                       1+3*nb_bits_base,ctrl_state=
                       base_to_ctrl_state(base_left_prev)+
                       base_to_ctrl_state(base_left)+
                       base_to_ctrl_state(base_right_matching_left_next)+
                       "0"
                   ),
                   [is_not_outside_mismatch]+
                   [sequence[k_left-1][i] for i in range(nb_bits_base)]+
                   [sequence[k_left  ][i] for i in range(nb_bits_base)]+
                   [right_matching_left_next[i] for i in range(nb_bits_base)]+
                   [energy[i] for i in range(nb_bits_energy)]
                  )
    return "outside mismatch"


def add_enclosing_mismatch_contribution(circuit,
                                        folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                                        all_registers,
                                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                                        all_nb_bits,
                                        incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                                        all_gates,
                                        length,sequence_specification,folding_specification,loop_specification,
                                        all_specifiers,
                                        basic_args_to_call_append,
                                        k_left,
                                        base_left_next,base_right_matching_left_prev,base_right_matching_left):
    #parameters base_ are in the order of the sequence
    will_use_registers(circuit,[folding,sequence,continue_searching_right_matching_left,nb_enclosed,right_matching_left_prev,use_right_matching_left_prev,energy])
    
    is_not_particular_case = continue_searching_right_matching_left
    base_left = base_matching_Watson_Crick(base_right_matching_left)
    
    contribution = mismatch_contribution(base_right_matching_left_prev,base_right_matching_left,base_left_next)
    
    if 0 < nb_bits_enclosed:
        #<****> what has changed is
        #base_left_next,base_right_matching_left_prev
        #U,U == 11,11 -> 00,11 == A,U
        #G,U == 10,11 -> 01,11 == C,U
        #C,U == 01,11 -> 10,11 == G,U
        #A,U == 00,11 -> 11,11 == U,U

        #U,C == 11,01 -> 00,01 == A,C
        #G,C == 10,01 -> 01,01 == C,C
        #C,C == 01,01 -> 10,01 == G,C
        #A,C == 00,01 -> 11,01 == U,C

        if "U" == base_left_next or "C" == base_left_next:
            if   "A" == base_right_matching_left_prev:
                base_right_matching_left_prev = "U"
            elif "C" == base_right_matching_left_prev:
                base_right_matching_left_prev = "G"
            elif "G" == base_right_matching_left_prev:
                base_right_matching_left_prev = "C"
            elif "U" == base_right_matching_left_prev:
                base_right_matching_left_prev = "A"
    
    circuit.append(add_cst_gate(contribution,nb_bits_energy).
                   control(
                       1+2*nb_bits_bracket_dot+1+3*nb_bits_base,ctrl_state=
                       base_to_ctrl_state(base_right_matching_left_prev)+
                       base_to_ctrl_state(base_left_next)+
                       base_to_ctrl_state(base_left)+

                       "1"+
                       bracket_dot_to_ctrl_state("<-.")+
                       bracket_dot_to_ctrl_state("(")+

                       "1"
                   ),
                   [is_not_particular_case]+
                   [folding[k_left  ][i] for i in range(nb_bits_bracket_dot)]+
                   [folding[k_left+1][i] for i in range(nb_bits_bracket_dot)]+
                   [use_right_matching_left_prev]+

                   [sequence[k_left  ][i] for i in range(nb_bits_base)]+
                   [sequence[k_left+1][i] for i in range(nb_bits_base)]+
                   [right_matching_left_prev[i] for i in range(nb_bits_base)]+
                   
                   [energy[i] for i in range(nb_bits_energy)]
                  )
    return "regular enclosing mismatch"


def stack_Watson_Crick_contribution(base_left5,base_left3):
    #the AU_end_penalty of the exterior base pair is added always (even if it is not an outside end)
    #and is substacted when an AU base pair is detected not being an outside end
    
    #so  if "A" or "U" == base_left5 then AU_end_penalty is added
    #and if "A" or "U" == base_left3 then AU_end_penalty is substracted
    
    return [
        #"A" == base_left5
        [-0.9               ,-2.2+AU_end_penalty,-2.1+AU_end_penalty,-1.1               ],
        #"C" == base_left
        [-2.1-AU_end_penalty,-3.3               ,-2.4               ,-2.1-AU_end_penalty],
        #"G" == base_left
        [-2.4-AU_end_penalty,-3.4               ,-3.3               ,-2.2-AU_end_penalty],
        #"U" == base_left
        [-1.3               ,-2.4+AU_end_penalty,-2.1+AU_end_penalty,-0.9               ]
    ][int(base_to_ctrl_state(base_left5),2)][int(base_to_ctrl_state(base_left3)    ,2)]


def add_stack_Watson_Crick_contribution(circuit,
                                        folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                                        all_registers,
                                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                                        all_nb_bits,
                                        incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                                        all_gates,
                                        length,sequence_specification,folding_specification,loop_specification,
                                        all_specifiers,
                                        basic_args_to_call_append,
                                        k_left,
                                        base_left,base_next_left):
    will_use_registers(circuit,[folding,sequence,continue_searching_right_matching_left,energy])
    
    is_stack_or_single_bulge = continue_searching_right_matching_left
    
    contribution = stack_Watson_Crick_contribution(base_left,base_next_left)
    
    circuit.append(add_cst_gate(contribution,nb_bits_energy).
                   control(
                       1+2*nb_bits_base,ctrl_state=
                       base_to_ctrl_state(base_next_left)+
                       base_to_ctrl_state(base_left)+
                       "1"
                   ),
                   [is_stack_or_single_bulge]+
                   [sequence[k_left  ][i] for i in range(nb_bits_base)]+
                   [sequence[k_left+1][i] for i in range(nb_bits_base)]+
                   
                   [energy[i] for i in range(nb_bits_energy)]
                  )
    return "Watson-Crick"


def add_loop_energy(circuit,
                    folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                    all_registers,
                    nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                    all_nb_bits,
                    incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                    all_gates,
                    length,sequence_specification,folding_specification,loop_specification,
                    all_specifiers,
                    basic_args_to_call_append,
                    k_left):
    will_use_registers(circuit,[folding,sequence,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,energy])
    
    #use https://rna.urmc.rochester.edu/NNDB/tutorials.html
    #sequence[0] is 5' --> sequence[length-1] is 3'
    
    bases = ["A","C","G","U"]
    
    
    # 1) loop_length
    
    #TODO
    #contributions of:
    # - loop_length
    # - asymmetry in interior loop
    # - missing AU_end_penalty on the exterior side of helixes
    # - AU_end_penalty is different in interior loops -> add the difference
    # - dandling end
    
    
    # 2) Hairpin
    
    is_hairpin_mismatch = continue_searching_right_matching_left
    
    #hairpin loop with 3 == loop_lengh do not have mismatch
    
    # 2.1) UU, GA ou AG interior mismatch
    # 2.1.1) Detect
    
    #U,U == 11,11 -> 11,11
    #G,A == 10,00 -> 11,01
    #A,G == 00,10 -> 01,11
    circuit.cx(sequence[k_left+1][1],sequence[k_left+1][0])
    circuit.cx(sequence[k_left+1][1],right_matching_left_prev[0])
    circuit.cx(right_matching_left_prev[1],sequence[k_left+1][0])
    circuit.cx(right_matching_left_prev[1],right_matching_left_prev[0])
    
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2             ,ctrl_state="11"  +n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][0],right_matching_left_prev[0],is_hairpin_mismatch])
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base,ctrl_state="0101"+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[is_hairpin_mismatch])
    
    #but hairpin loop with 3 >= loop_lengh do not have mismatch
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base+(nb_bits_position-2),ctrl_state=n_to_ctrl_state(0,nb_bits_position-2)+"0101"+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[loop_length[i] for i in range(2,nb_bits_position)]+[is_hairpin_mismatch])
    
    
    # 2.1.2) Apply contribution
    
    circuit.append(add_cst_gate(hairpin_mismatch_UU_AG_contribution,nb_bits_energy).control(),[is_hairpin_mismatch]+[energy[i] for i in range(nb_bits_energy)])
    
    
    # 2.1.3) Restore
    
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base+(nb_bits_position-2),ctrl_state=n_to_ctrl_state(0,nb_bits_position-2)+"0101"+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[loop_length[i] for i in range(2,nb_bits_position)]+[is_hairpin_mismatch])
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base,ctrl_state="0101"+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[is_hairpin_mismatch])
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2             ,ctrl_state="11"  +n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][0],right_matching_left_prev[0],is_hairpin_mismatch])
    circuit.cx(right_matching_left_prev[1],right_matching_left_prev[0])
    circuit.cx(right_matching_left_prev[1],sequence[k_left+1][0])
    circuit.cx(sequence[k_left+1][1],right_matching_left_prev[0])
    circuit.cx(sequence[k_left+1][1],sequence[k_left+1][0])
    
    
    # 2.2) GG interior mismatch
    
    #detect
    is_hairpin_mismatch_GG = continue_searching_right_matching_left
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base,ctrl_state=base_to_ctrl_state("G")+base_to_ctrl_state("G")+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[is_hairpin_mismatch])
    
    #but hairpin loop with 3 >= loop_lengh do not have mismatch
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base+(nb_bits_position-2),ctrl_state=n_to_ctrl_state(0,nb_bits_position-2)+base_to_ctrl_state("G")+base_to_ctrl_state("G")+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[loop_length[i] for i in range(2,nb_bits_position)]+[is_hairpin_mismatch])
    
    #apply contribution
    circuit.append(add_cst_gate(hairpin_mismatch_GG_contribution,nb_bits_energy).control(),[is_hairpin_mismatch]+[energy[i] for i in range(nb_bits_energy)])
    
    #restore
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base+(nb_bits_position-2),ctrl_state=n_to_ctrl_state(0,nb_bits_position-2)+base_to_ctrl_state("G")+base_to_ctrl_state("G")+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[loop_length[i] for i in range(2,nb_bits_position)]+[is_hairpin_mismatch])
    circuit.append(MCXGate(nb_bits_bracket_dot+nb_bits_enclosed+2*nb_bits_base,ctrl_state=base_to_ctrl_state("G")+base_to_ctrl_state("G")+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[is_hairpin_mismatch])
    
    
    # 2.3) C3 hairpin loop
    
    circuit.append(add_cst_gate(hairpin_C3_contribution,nb_bits_energy).control(nb_bits_bracket_dot+nb_bits_enclosed+(nb_bits_position-2)+3*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+base_to_ctrl_state("C")+n_to_ctrl_state(0,nb_bits_position-2)+n_to_ctrl_state(0,nb_bits_enclosed)+bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[loop_length[i] for i in range(2,nb_bits_position)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[sequence[k_left+2][i] for i in range(nb_bits_base)]+[sequence[k_left+3][i] for i in range(nb_bits_base)]+[energy[i] for i in range(nb_bits_energy)])
    
    
    # 2.4) all-C hairpin loop with 3 < loop_length (ignored)
    
    
    
    if 0 < nb_bits_enclosed:
        # 3) Multibranch loop
        
        #use https://rna.urmc.rochester.edu/NNDB/turner04/mb.html
        
        
        if 1 < nb_bits_enclosed:
            # 3.1) Constant

            #<*****> energy has been initialised to (length-1-min_length_hairpin_loop)*multiloop_flat_contribution
            #so multiloop_flat_contribution should be substracted if there is no multiloop starting at position k_left
            circuit.append(substract_cst_gate(multiloop_flat_contribution,nb_bits_energy).control(nb_bits_enclosed-1,ctrl_state=n_to_ctrl_state(0,nb_bits_enclosed-1)),[nb_enclosed[i] for i in range(1,nb_bits_enclosed)]+[energy[i] for i in range(nb_bits_energy)])
        
        
        # 3.2) Average asmmetry (ignored)
        
        
        # 3.3) Number of branches
        
        #add nb_enclosed*one_branch_contribution to energy
        for i in range(nb_bits_enclosed):
            for _ in range(2**i):
                circuit.append(add_cst_gate(one_branch_contribution,nb_bits_energy).control(),[nb_enclosed[i]]+[energy[i] for i in range(nb_bits_energy)])

        #a loop that is not a multiloop may have 1 enclosed branch, so this contribution should be cancelled
        circuit.append(substract_cst_gate(one_branch_contribution,nb_bits_energy).control(nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)),[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[energy[i] for i in range(nb_bits_energy)])
        
        
        # 3.4) Particular case of strain (three-way branching loops with fewer than two unpaired nucleotides)
        
        #<***> control by 2 > loop_length
        circuit.append(add_cst_gate(strain_contribution,nb_bits_energy).control(nb_bits_enclosed+(nb_bits_position-1),ctrl_state=n_to_ctrl_state(0,nb_bits_position-1)+n_to_ctrl_state(2,nb_bits_enclosed)),[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[loop_length[i] for i in range(1,nb_bits_position)]+[energy[i] for i in range(nb_bits_energy)])
    
    
        # 4) Stack and single nucleotide bulge loop

        #use https://rna.urmc.rochester.edu/NNDB/turner04/wc.html
        #and https://rna.urmc.rochester.edu/NNDB/turner04/bulge.html
        #this RNA is not a duplex, so there is no symmetric duplex term
        #(if this is a misunderstanding, then it is actually a simplification choice!)
        #the absence of AU_end_penalty for single nucleotide bulge loop has not been forgotten


        # 4.1) Stack

        #use continue_searching_right_matching_left as an auxillary qbit
        #(that should be cleaned after use)
        is_stack_or_single_bulge = continue_searching_right_matching_left
        circuit.append(MCXGate(nb_bits_enclosed+(nb_bits_position-1),ctrl_state=n_to_ctrl_state(0,nb_bits_position-1)+n_to_ctrl_state(1,nb_bits_enclosed)),[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[loop_length[i] for i in range(1,nb_bits_position)]+[is_stack_or_single_bulge])
        for base_left in bases:
            for base_next_left in bases:
                add_stack_Watson_Crick_contribution(*basic_args_to_call_append,k_left,base_left,base_next_left)

        #restore is_stack_or_single_bulge
        circuit.append(MCXGate(nb_bits_enclosed+(nb_bits_position-1),ctrl_state=n_to_ctrl_state(0,nb_bits_position-1)+n_to_ctrl_state(1,nb_bits_enclosed)),[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[loop_length[i] for i in range(1,nb_bits_position)]+[is_stack_or_single_bulge])


        # 4.2) Special C bulge
        # 4.2.1) Detect

        #a special C bulge is a "bulged C [residue] adjacent to at least one C" (from https://www.pnas.org/doi/10.1073/pnas.0401799101)

        is_special_C = continue_searching_right_matching_left

        #C == sequence[k_left]
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left  ][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        #C == sequence[k_left+2]
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left+2][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        #correct C == sequence[k_left] and C == sequence[k_left+2] redundancy
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+3*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[sequence[k_left+2][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])

        #G == sequence[k_left] (so C == right_matching_left)
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        #G == next_left (so C == right_matching_next_left)
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[next_left[i       ] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        #correct #G == sequence[k_left] and G == next_left redundancy
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+3*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[next_left[i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])


        # 4.2.2) Apply contribution

        circuit.append(add_cst_gate(special_C_bulge_bonus,nb_bits_energy).control(),[is_special_C]+[energy[i] for i in range(nb_bits_energy)])


        # 4.2.3) Restore

        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+3*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[next_left[i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[next_left[i       ] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("G")+bracket_dot_to_ctrl_state("|")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[right_matching_left_prev[i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+3*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left][i] for i in range(nb_bits_base)]+[sequence[k_left+2][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left+2][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])
        circuit.append(MCXGate(nb_bits_enclosed+nb_bits_position+1+2*nb_bits_base,ctrl_state=base_to_ctrl_state("C")+base_to_ctrl_state("C")+bracket_dot_to_ctrl_state(".")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_enclosed)),[sequence[k_left  ][i] for i in range(nb_bits_base)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[folding[k_left+1][0]]+[loop_length[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_special_C])


        # 4.3) Degeneracy of bulge (ignored)

    
    # 5) Flat contribution per helix
    
    #for helix_flat_contribution = 4.09
    #use the intermolecular initiation term from https://rna.urmc.rochester.edu/NNDB/turner04/wc.html
    
    
    # 5.1) Detect an end of helix
    
    #use continue_searching_right_matching_left as an auxillary qbit
    #(that should be cleaned after use)
    is_helix_interior_end = continue_searching_right_matching_left
    
    #set is_helix_interior_end = 2 > nb_enclosed and 0 == loop_length
    if 1 < nb_bits_enclosed:
        circuit.append(MCXGate((nb_bits_enclosed-1)+nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)+n_to_ctrl_state(0,nb_bits_enclosed-1)),[nb_enclosed[i] for i in range(1,nb_bits_enclosed)]+[loop_length[i] for i in range(nb_bits_position)]+[is_helix_interior_end])
    else:
        #2 > nb_enclosed is true
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[loop_length[i] for i in range(nb_bits_position)]+[is_helix_interior_end])
    
    #and flip the result to get is_helix_interior_end = 2 <= nb_enclosed or 0 < loop_length
    circuit.x(is_helix_interior_end)
    
    
    # 5.2) Add the contribution
    
    circuit.append(add_cst_gate(helix_flat_contribution,nb_bits_energy).control(),[is_helix_interior_end]+[energy[i] for i in range(nb_bits_energy)])
    
    
    # 5.3) Restore is_helix_interior_end
    
    if 1 < nb_bits_enclosed:
        circuit.append(MCXGate((nb_bits_enclosed-1)+nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)+n_to_ctrl_state(0,nb_bits_enclosed-1)),[nb_enclosed[i] for i in range(1,nb_bits_enclosed)]+[loop_length[i] for i in range(nb_bits_position)]+[is_helix_interior_end])
    else:
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[loop_length[i] for i in range(nb_bits_position)]+[is_helix_interior_end])
    
    #continue_searching_right_matching_left has been restored
    #except that now 1 == continue_searching_right_matching_left
    
    
    if 0 < nb_bits_enclosed:
        # 6) substract loop_length_right to loop_length

        #use continue_searching_right_matching_left as an auxillary qbit
        #(that should be cleaned after use)
        #initially 1 == continue_searching_right_matching_left
        initial_carry=continue_searching_right_matching_left
        circuit.x(loop_length_right)
        circuit.append(add_position,[initial_carry]+[loop_length_right[i] for i in range(nb_bits_position)]+[loop_length[i] for i in range(nb_bits_position)])

        #now loop_length is used as loop_length_left
        loop_length_left = loop_length



        # 7) Other tow-branched loop (bulge and interior loop)

        #these are loops with 1 == nb_enclosed and 0 < loop_length

        should_not_cancel_mismatch = continue_searching_right_matching_left


        # 7.1) Particular cases 1*1 interior loop
        # 7.1.1) Cancel potential contribution that will come from 8.1) with a greater k_left

        #detect
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed+(nb_bits_bracket_dot-1)+1,ctrl_state="01"+n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[folding[k_left+1][nb_bits_bracket_dot-1],use_right_matching_left_prev,should_not_cancel_mismatch])

        for base_next_left_prev in bases:
            for base_next_left in bases:
                for base_right_matching_next_left_next in bases:
                    substract_enclosed_mismatch_contribution(*basic_args_to_call_append,k_left,base_next_left_prev,base_next_left,base_right_matching_next_left_next)

        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed+(nb_bits_bracket_dot-1)+1,ctrl_state="01"+n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[folding[k_left+1][nb_bits_bracket_dot-1],use_right_matching_left_prev,should_not_cancel_mismatch])

        # 7.1.2) Contribution of these particular cases

        is_not_11 = continue_searching_right_matching_left

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_11])

        for base_left in bases:
            for base_interior_left in bases:
                for base_next_left in bases:
                    for base_interior_right in bases:
                        #TODO: easy but boring
                        #use https://rna.urmc.rochester.edu/NNDB/turner04/int11.txt
                        pass

        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_11])


        # 7.2) Particular cases 1*2 and 2*1 interior loop
        # 7.2.1) Cancel potential contribution that will come from 8.1) with a greater k_left

        #detect

        #make 1*2 look like 2*1
        #when 2*1
        #   <-. == folding[k_left+1]
        #   0 == folding[k_left+1][1]
        #   this conserves use_right_matching_left_next
        #when 1*2
        #   1 == use_right_matching_left_next
        #   this sets use_right_matching_left_next=folding[k_left+1][1]
        #then to use use_right_matching_left_next for the **enclosed** (not the enclosing) mismatch,
        #it should be used negatively controlled
        circuit.cx(folding[k_left+1][1],use_right_matching_left_next,ctrl_state=0)

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed+1,ctrl_state="0"+n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[use_right_matching_left_prev,should_not_cancel_mismatch])

        for base_next_left_prev in bases:
            for base_next_left in bases:
                for base_right_matching_next_left_next in bases:
                    substract_enclosed_mismatch_contribution(*basic_args_to_call_append,k_left,base_next_left_prev,base_next_left,base_right_matching_next_left_next)


        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed+1,ctrl_state="0"+n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[use_right_matching_left_prev,should_not_cancel_mismatch])
        circuit.cx(folding[k_left+1][1],use_right_matching_left_next,ctrl_state=0)


        # 7.2.2) Contribution of 1*2 particular cases

        is_not_12 = continue_searching_right_matching_left

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(2,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_12])

        for base_left in bases:
            for base_interior_left in bases:
                for base_next_left in bases:
                    for base_interior_right5 in bases:
                        for base_interior_right3 in bases:
                            #TODO: easy but boring and long
                            #use https://rna.urmc.rochester.edu/NNDB/turner04/int21.txt
                            pass

        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(2,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_12])


        # 7.2.3) Contribution of 2*1 particular cases

        is_not_21 = continue_searching_right_matching_left

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_21])

        for base_left in bases:
            for base_interior_left5 in bases:
                for base_interior_left3 in bases:
                    for base_next_left in bases:
                        for base_interior_right in bases:
                            #TODO: easy but boring and long
                            #use https://rna.urmc.rochester.edu/NNDB/turner04/int21.txt
                            #could probably be merged with 7.2.2)
                            pass

        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_21])


        # 7.3) Particular cases 2*2 interior loop
        # 7.3.1) Cancel potential contribution that will come from 8.1) with a greater k_left

        is_not_22 = should_not_cancel_mismatch

        #detect
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_22])

        for base_next_left_prev in bases:
            for base_next_left in bases:
                for base_right_matching_next_left_next in bases:
                    substract_enclosed_mismatch_contribution(*basic_args_to_call_append,k_left,base_next_left_prev,base_next_left,base_right_matching_next_left_next)


        # 7.3.2) Contribution of these particular cases

        for base_left in bases:
            for base_interior_left5 in bases:
                for base_interior_left3 in bases:
                    for base_next_left in bases:
                        for base_interior_right5 in bases:
                            for base_interior_right3 in bases:
                                #TODO: easy but boring and very long
                                #use https://rna.urmc.rochester.edu/NNDB/turner04/int22.txt
                                pass

        #restore
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_enclosed,ctrl_state=n_to_ctrl_state(1,nb_bits_enclosed)+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)]+[is_not_22])

    
    # 8) Mismatch
    # 8.1) Enclosed (particular cases compensated in 7.x.1))
    
    #let's consider the mismatch outside of the loop enclosed by ( == folding[k_left]
    #add its contribution whatever the case
    #and if it happends it was a particular case of enclosed mismatch
    #then it has been corrected when this loop was treated in 3) at a different k_left
    
    #folding[k_left] == ( followed by ...)<-. is the shortest way to get a mismatch outside of a loop
    if 0 < k_left and k_left < length-2-min_length_hairpin_loop:
        
        is_not_outside_mismatch = continue_searching_right_matching_left
        
        circuit.append(MCXGate(2*nb_bits_bracket_dot+1,ctrl_state="1"+bracket_dot_to_ctrl_state("(")+bracket_dot_to_ctrl_state(".->")),[folding[k_left-1][i] for i in range(nb_bits_bracket_dot)]+[folding[k_left  ][i] for i in range(nb_bits_bracket_dot)]+[use_right_matching_left_next,is_not_outside_mismatch])
        
        for base_left_prev in bases:
            for base_left in bases:
                for base_right_matching_left_next in bases:
                    add_outside_mismatch_contribution(*basic_args_to_call_append,k_left,base_left_prev,base_left,base_right_matching_left_next)
    
    
    # 8.2) Enclosing (no particular cases)
    
    #there is no particular case if there is only the exterior loop and maybe an hairpin loop
    if 0 < nb_bits_enclosed:
        # 8.2.1) Detect non particular cases

        #use continue_searching_right_matching_left as an auxillary qbit
        #(that should be cleaned after use)
        #initially 1 == continue_searching_right_matching_left
        is_not_particular_case = continue_searching_right_matching_left

        #1*_ interior loop are particular cases
        #there is no mismatch in bulges (so do not care about _*0)
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[is_not_particular_case])

        #_*1 interior loop are particular cases
        #there is no mismatch in bulges (so do not care about 0*_)
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(1,nb_bits_position)),[loop_length_right[i] for i in range(nb_bits_position)]+[is_not_particular_case])

        #before correcting for redundancy with 1*1
        #make 2*2 and 1*1 look like 1*1 over one less qbit like this
        #0 -> 0
        #1 -> 3
        #2 -> 2
        #3 -> 1
        circuit.cx(loop_length_left[0],loop_length_left[1])
        circuit.cx(loop_length_right[0],loop_length_right[1])

        #correct redundancy with 1*1
        #and handle 2*2 that are also particular cases
        circuit.append(MCXGate(2*(nb_bits_position-1),ctrl_state=n_to_ctrl_state(1,nb_bits_position-1)+n_to_ctrl_state(1,nb_bits_position-1)),[loop_length_left[i] for i in range(1,nb_bits_position)]+[loop_length_right[i] for i in range(1,nb_bits_position)]+[is_not_particular_case])

        #2*3 with GA or GG mismatch are particualr cases
        #but 2*3 -> 2*1 now
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="0"+base_to_ctrl_state("G")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(2,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[0],is_not_particular_case])

        #3*2 with GA or GG mismatch are particular cases
        #but 3*2 -> 1*2 now
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="0"+base_to_ctrl_state("G")+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[0],is_not_particular_case])

        #A,G == 00,10 -> 00,10
        #U,U == 11,11 -> 00,11
        #in order to control by 00,1x
        for i in range(nb_bits_bracket_dot):
            circuit.cx(right_matching_left_prev[0],sequence[k_left+1][i])

        #so what has changed is
        #U,U == 11,11 -> 00,11 == A,U
        #G,U == 10,11 -> 01,11 == C,U
        #C,U == 01,11 -> 10,11 == G,U
        #A,U == 00,11 -> 11,11 == U,U

        #U,C == 11,01 -> 00,01 == A,C
        #G,C == 10,01 -> 01,01 == C,C
        #C,C == 01,01 -> 10,01 == G,C
        #A,C == 00,01 -> 11,01 == U,C

        #it will be used there <****>

        #2*3 with UU or AG mismatch are particular cases
        #but 2*3 -> 2*1 now
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="001"+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(2,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[right_matching_left_prev[0]]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[is_not_particular_case])

        #3*2 with UU or AG mismatch are particular cases
        #but 3*2 -> 1*2 now
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="001"+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[right_matching_left_prev[0]]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[is_not_particular_case])
    
    
    # 8.2.2) Handle non particular cases
    
    for base_left_next in bases:
        for base_right_matching_left_prev in bases:
            for base_right_matching_left in bases:
                add_enclosing_mismatch_contribution(*basic_args_to_call_append,k_left,base_left_next,base_right_matching_left_prev,base_right_matching_left)
    
    
    if 0 < nb_bits_enclosed:
        # 8.2.3) Restore everything

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="001"+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[right_matching_left_prev[0]]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[is_not_particular_case])
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="001"+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(2,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[right_matching_left_prev[0]]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[is_not_particular_case])

        #OPTIMISATION: restoring this could be avoided by maintaining a global variable about permutation of base encoding
        #00,10 -> 00,10
        #11,11 -> 00,11
        for i in range(nb_bits_bracket_dot):
            circuit.cx(right_matching_left_prev[0],sequence[k_left+1][i])

        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="0"+base_to_ctrl_state("G")+n_to_ctrl_state(2,nb_bits_position)+n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[0],is_not_particular_case])
        circuit.append(MCXGate(2*nb_bits_position+nb_bits_bracket_dot+(nb_bits_bracket_dot-1),ctrl_state="0"+base_to_ctrl_state("G")+n_to_ctrl_state(1,nb_bits_position)+n_to_ctrl_state(2,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[loop_length_right[i] for i in range(nb_bits_position)]+[sequence[k_left+1][i] for i in range(nb_bits_base)]+[right_matching_left_prev[0],is_not_particular_case])
        circuit.append(MCXGate(2*(nb_bits_position-1),ctrl_state=n_to_ctrl_state(1,nb_bits_position-1)+n_to_ctrl_state(1,nb_bits_position-1)),[loop_length_left[i] for i in range(1,nb_bits_position)]+[loop_length_right[i] for i in range(1,nb_bits_position)]+[is_not_particular_case])
        circuit.cx(loop_length_right[0],loop_length_right[1])
        circuit.cx(loop_length_left[0],loop_length_left[1])
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(1,nb_bits_position)),[loop_length_right[i] for i in range(nb_bits_position)]+[is_not_particular_case])
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(1,nb_bits_position)),[loop_length_left[i] for i in range(nb_bits_position)]+[is_not_particular_case])
    
    
    circuit.x(continue_searching_right_matching_left)
    
    return "loop energy" 


Analysis of `folding` (without looking at `sequence`, only copying some part of it for use by the `energy` comutation).

In [266]:
def automaton_step(circuit,
                   folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                   all_registers,
                   nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                   all_nb_bits,
                   incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                   all_gates,
                   length,sequence_specification,folding_specification,loop_specification,
                   all_specifiers,
                   basic_args_to_call_append,
                   k_left,k,
                   compute_energy=True,
                   uncompute=False):
    if compute_energy:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next])
        if not uncompute:
            will_use_registers(circuit,[incorrect_configuration])
    else:
        will_use_registers(circuit,[folding,count,continue_searching_right_matching_left])
    
    #<*> if it has been checked that there is a matching ) to find,
    #then continue_searching_right_matching_left == 1
    #otherwise continue_searching_right_matching_left == 0
    
    
    #1), 2) and 3) before 4) Update count
    #and
    #5) and 6) after 4) Update count
    #so that things are always controled by 1 == count
    #(which could be used by the compiler for optimisation when implementing sevral time in a row the same MCXGate())
    
    if compute_energy:
        
        
        # 1) Get next_left_prev and next_left
        
        for i in range(nb_bits_base):
            #if 0 < k (which is always the case when compute_energy)
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k-1][i],next_left_prev[i]])
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],next_left     [i]])
        
        
        # 2) Get right_matching_left_prev and right_matching_left_next
        
        #get right_matching_left_prev and use_right_matching_left_prev
        for i in range(nb_bits_base):
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1                  ,ctrl_state="1"                             +n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k-1][i],right_matching_left_prev[i]])
        circuit.append(        MCXGate(1+nb_bits_bracket_dot+nb_bits_position+nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state(".->")+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[folding[ k-1][i] for i in range(nb_bits_bracket_dot)]+[use_right_matching_left_prev])
        
        #FUTURE: if one adds GU pairs, then one base of a valid base pair would not be enough anymore to know which base pair it is
        #so there would be a need for right_matching_left, this way:
        #for i in range(nb_bits_base):
        #    circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1                     ,ctrl_state="1"                              +n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],right_matching_left[i]     ])
        
        #get right_matching_left_next and use_right_matching_left_next
        if length-1 > k:
            for i in range(nb_bits_base):
                circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1                  ,ctrl_state="1"                             +n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k+1][i],right_matching_left_next[i]])
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state("<-.")+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[folding[ k+1][i] for i in range(nb_bits_bracket_dot)]+[use_right_matching_left_next])
        #else use_right_matching_left_next=0
        
        
        # 3) Check base pairing rules
        
        #FUTURE: if one adds GU pairs, then there would be a right_matching_left
        #so base pairing rules could be checked more efficiently in characterise_loop()
        
        #the base encoding is such that cx(sequence[k_left],sequence[k_right]) leads to
        #"11" == sequence[k_right] if and only if (sequence[k_left],sequence[k_right]) is a valid Watson-Crick base pair
        for i in range(nb_bits_base):
            circuit.cx(sequence[k_left][i],sequence[k][i])
        if not uncompute:
            circuit.append(incr_incorrect.control(1+nb_bits_bracket_dot+nb_bits_position+nb_bits_base,ctrl_state="11"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[sequence[k][i] for i in range(nb_bits_base)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        #there is actually no need to restore sequence[k]
    
    
    # 4) Update count
    
    #if 1 == continue_searching_right_matching_left
    #then update count for the next candidate
    #OPTIMISATION: use smaller increment and decrement circuits when the maximum value of count enables it (since it never goes negative)
    if 0 == k:
        #then -1 == k_left and ( == folding[k=-1]
        # - so 1 == continue_searching_right_matching_left
        #   so it is ok not controlling by continue_searching_right_matching_left here
        # - so 1 == count
        #   so there is no need to incr_position or decr_position in order to increment or decrement
        circuit.append(MCXGate((nb_bits_bracket_dot-1),ctrl_state=bracket_dot_to_ctrl_state("|")),[folding[k][0]]+[count[0]])
        circuit.append(MCXGate( nb_bits_bracket_dot   ,ctrl_state=bracket_dot_to_ctrl_state("(")),[folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[1]])
    else:
        circuit.append(incr_position.control(1+nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
        circuit.append(decr_position.control(1+nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
    
    #if 0 == continue_searching_right_matching_left then simply increment count so that it never equals 0 again
    circuit.append(incr_position.control(ctrl_state="0"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)])
    
    
    if compute_energy:
        
        
        # 5) Get right_matching_next_left_next
        
        if k < length-1:
            for i in range(nb_bits_base):
                #FUTURE: if one adds GU pairs, then one base of a valid base pair is not enough anymore to know which base pair it is
                #so there would be a need for right_matching_next_left, this way:
                #circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],right_matching_next_left[i]     ])
                circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k+1][i],right_matching_next_left_next[i]])
        
        
        # 6) Update nb_enclosed
        
        if 0 < nb_bits_enclosed:
            #OPTIMISATION: use smaller increment circuit when the maximum value of nb_enclose enables it (which depends on min_length_hairpin_loop)
            circuit.append(incr_nb_enclosed.control(1+nb_bits_bracket_dot+nb_bits_position,ctrl_state=n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)])
        
        
        # 7) Update loop_length
        
        #loop_length is the number of . for which 1 == count
        
        #if 1 == continue_searching_right_matching_left and 1 == count and "x0" == . == folding[k]
        #then increment loop_length
        #OPTIMISATION: use smaller increment circuit when the maximum value of loop_length enables it
        if k_left+1 == k:
            #since the initial value 1 == loop_length is known then there is no need to incr_position in order to increment
            #two CXGate() are sufficient
            circuit.append(MCXGate(              (nb_bits_bracket_dot-1)+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,loop_length[0]])
            circuit.cx(loop_length[0],loop_length[1],ctrl_state="0")
        else:
            circuit.append(incr_position.control((nb_bits_bracket_dot-1)+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[loop_length[i] for i in range(nb_bits_position)])
        
        
        # 8) Update loop_length_right
        
        #in case of a non multi-loop,
        #(that is 1 => nb_enclosed)
        #loop_length_right is the number of . for which 1 == count after the first enclosed stem (if any)
        
        #if 1 == continue_searching_right_matching_left and 1 == count and "x0" == . == folding[k]
        #and 1 == nb_enclosed (that is 1 == nb_enclosed[0] since 1 => nb_enclosed)
        #then increment loop_length_right
        #OPTIMISATION: use smaller increment circuit when the maximum possible value of loop_length enables it
        if k_left+2+min_length_hairpin_loop < k and 0 < nb_bits_enclosed:
            circuit.append(incr_position.control((nb_bits_bracket_dot-1)+nb_bits_position+2,ctrl_state="11"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,nb_enclosed[0]]+[loop_length_right[i] for i in range(nb_bits_position)])
    
    
    # 9) Update continue_searching_right_matching_left

    #if 0 == count
    #then there is no more matching ) to find
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])
    
    return "step"


def characterise_loop(circuit,
                      folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                      all_registers,
                      nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                      all_nb_bits,
                      incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                      all_gates,
                      length,sequence_specification,folding_specification,loop_specification,
                      all_specifiers,
                      basic_args_to_call_append,
                      k_left,
                      compute_energy=True,uncompute=False):
    if compute_energy:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration])
    else:
        will_use_registers(circuit,[folding,count,continue_searching_right_matching_left])
    
    
    # 1) Initialisation of the automaton at the position k_left
    
    #initialise count with folding[k_left]
    #folding[k_left] -> count
    #. == "x0"       -> 0 == "0...000"
    #do nothing
    #) == "11"       -> 0 == "0...000"
    #do nothing
    #( == "01"       -> 1 == "0...001"
    if 0 <= k_left:
        circuit.ccx(folding[k_left][0],folding[k_left][1],count[0],ctrl_state=bracket_dot_to_ctrl_state("("))
    else:
        circuit.x(count[0])
    
    #initialise continue_searching_right_matching_left=1 when 1 == count
    #indeed, when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.cx(count[0],continue_searching_right_matching_left)
    
    
    # 2) Compute parameters by scanning the configuration with the automaton
    
    for k in range(k_left+1,length):
        #OPTIMISATION: break the automaton into sevral automatons to run sequencially in order to reduce the number of qbits
        automaton_step(*basic_args_to_call_append,k_left,k,compute_energy,uncompute)
    
    #Computed parameters
    #folding[k_left] -> continue_searching_right_matching_left * nb_enclosed * loop_length
    #(               -> 0 if a matching ) has been found       * nb_enclosed * number of . (at level 1) in the loop
    #(               -> 1 if a matching ) has not been found   * unspecified * unspecified
    #. or )          -> 0                                      * 0           * 0

    #and:
    #loop_length_right
    #next_left_prev
    #next_left
    #right_matching_next_left_next
    #right_matching_left_prev
    #right_matching_left_next
    
    return "characterise"

Initialisations and supervision of the anaysis of `folding` and the computation of `energy` that is based on it.

In [267]:
def initialise_sequence(circuit,
                        folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                        all_registers,
                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                        all_nb_bits,
                        incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                        all_gates,
                        length,sequence_specification,folding_specification,loop_specification,
                        all_specifiers,
                        basic_args_to_call_append):
    will_use_registers(circuit,[sequence])
    
    #initialise according to a sequence_specification
    
    for k_left in range(length):
        if ")" != folding_specification[k_left]:
            base_specification=sequence_specification[k_left]
            #N == A or C or G or U
            if "N" == base_specification or "." == base_specification:
                circuit.h(sequence[k_left])
                pass
            elif "A" == base_specification:
                #A == "00"
                #do nothing
                pass
            #C
            elif "C" == base_specification:
                #C == "01"
                circuit.x(sequence[k_left][0])
            #G
            elif "G" == base_specification:
                #G == "10"
                circuit.x(sequence[k_left][1])
            #U
            elif "U" == base_specification:
                #U == "11"
                circuit.x(sequence[k_left])
            #R == A or G
            elif "R" == base_specification:
                circuit.h(sequence[k_left][1])
            #Y == C or U
            elif "Y" == base_specification:
                circuit.x(sequence[k_left][0])
                circuit.h(sequence[k_left][1])
            #S == G or C
            elif "S" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.cx(sequence[k_left][0],sequence[k_left][1],ctrl_state="0")
            #W == A or U
            elif "W" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.cx(sequence[k_left][0],sequence[k_left][1])
            #K == G or U
            elif "K" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.x(sequence[k_left][1])
            #M == A or C
            elif "M" == base_specification:
                circuit.h(sequence[k_left][0])
            #B == C or G or U
            else:
                change_sequence_encoding = QuantumRegister(1,name=f"change_seq_{k_left}")
                will_use_registers(circuit,[change_sequence_encoding])
                if "B" == base_specification:
                    #A == "00" -> C == "01"
                    circuit.h(sequence[k_left])
                    circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding,ctrl_state="00")
                    circuit.cx(change_sequence_encoding,sequence[k_left][0])
                    #AMPLITUDE: the amplitude of C is now twice larger than the amplitude of G or the amplitude of U
                #D == A or G or U
                elif "D" == base_specification:
                    #C == "01" -> A == "00"
                    circuit.h(sequence[k_left])
                    circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding,ctrl_state="01")
                    circuit.cx(change_sequence_encoding,sequence[k_left][0])
                    #AMPLITUDE: the amplitude of A is now twice larger than the amplitude of G or the amplitude of U
                #H == A or C or U
                elif "H" == base_specification:
                    #G == "10" -> C == "11"
                    circuit.h(sequence[k_left])
                    circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding,ctrl_state="10")
                    circuit.cx(change_sequence_encoding,sequence[k_left][0])
                    #AMPLITUDE: the amplitude of C is now twice larger than the amplitude of A or the amplitude of U
                #V == A or C or G
                elif "V" == base_specification:
                    #U == "11" -> G == "10"
                    circuit.h(sequence[k_left])
                    circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding,ctrl_state="11")
                    circuit.cx(change_sequence_encoding,sequence[k_left][0])
                    #AMPLITUDE: the amplitude of G is now twice larger than the amplitude of A or the amplitude of C
                else:
                    raise Exception(f"Wrong sequence specification format: \"{sequence_specification[k_left]}\" found at index {k_left} in sequence_specification.")
            
            #only consider sequences that would respect the base pairing rules with the folding_specification
            if "(" == folding_specification[k_left]:
                for i in range(nb_bits_base):
                    #loop_specification[k_left][0][3] is
                    #                             [3] the position
                    #                          [0] of the matching )
                    #                  [k_left] of ( == folding_specification[k_left]
                    circuit.cx(sequence[k_left][i],sequence[loop_specification[k_left][0][3]][i],ctrl_state="0")
    
    return "init_sequence"


def initialise_folding(circuit,
                       folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
                       all_registers,
                       nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                       all_nb_bits,
                       incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                       all_gates,
                       length,sequence_specification,folding_specification,loop_specification,
                       all_specifiers,
                       basic_args_to_call_append):
    will_use_registers(circuit,[folding])
    #any bracket-dot symbol at any position
    for k in range(length):
        circuit.h(folding[k])
    
    #imagine a position -1, where folding[k_left=-1] == ( to check if the folding is well formed
    
    return "init_folding"


def RNA_design(circuit,
               folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
               all_registers,
               nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
               all_nb_bits,
               incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
               all_gates,
               length,sequence_specification,folding_specification,loop_specification,
               all_specifiers,
               basic_args_to_call_append):
    will_use_registers(circuit,all_registers)
    
    
    # 0) Ad hoc initialisation
    
    #initialise loop_length=1 because:
    # - then too small hairpin loop_length will be loop_length > 4 == 1+min_length_hairpin_loop so it is easy to check there <**>
    # - when doing it at the beginning, it is done in parallele whith other initialisation
    circuit.x(loop_length[0])
    
    #initialise energy to (length-1-min_length_hairpin_loop)*multiloop_flat_contribution
    #it is used there <*****>
    #TODO
    
    # 1) Superposition at the beginning of the Grover search
    
    initialise_folding(*basic_args_to_call_append)
    initialise_sequence(*basic_args_to_call_append)
    
    
    # 2) Check ) matching
    
    #check if there is a folding[k_right] == ) non matched with a folding[k_left] == (
    #that would then match with a ( at an imaginary position folding[k_left=-1]
    #the result is in continue_searching_right_matching_left:
    # 1 == continue_searching_right_matching_left if ( == folding[k_left=-1] has not been matched
    # 0 == continue_searching_right_matching_left otherwise
    
    characterise_loop(*basic_args_to_call_append,-1,False,False)
    
    #increment incorrect_configuration if necessary
    #(it is necessary to increment since the value of incorrect_configuration is unknown since characterise_loop(uncompute=False) can have changed it)
    circuit.append(incr_incorrect.control(ctrl_state="0"),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])

    inversed(characterise_loop)(*basic_args_to_call_append,-1,False,True)
    
    
    # 3) Check ( matching and add energ of loop
    
    for k_left in range(length-1-min_length_hairpin_loop):
        
        
        # 3.1) Compute parameters
        
        characterise_loop(*basic_args_to_call_append,k_left)
        
        
        # 3.2) Check ( matching
        
        #increment incorrect_configuration if folding[k_left] == ( has no matching )
        circuit.append(incr_incorrect.control(),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #TODO: set nb_bits_incorrect (with respect to the following comment)
        #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
        #remark that if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0)
        #then there is a ) somewhere
        #so continue_searching_right_matching_left[k_left=somewhere] == 0
        #so incorrect_folding is not incremented at each position
        #so the sum cannot overflow
        
        
        #continue_searching_right_matching_left has been used
        #and it can be considered that 0 == continue_searching_right_matching_left
        #(don't care about the energy if 1 == continue_searching_right_matching_left since 0 != incorrect_configuration)
        #so it can be used as an auxillary qbit
        #as far as it is cleaned after use (since inversed(characterise_loop) will be applied at the end)
        
        
        # 3.3) Check hairpin steric constraint
        
        #<**> too small hairpin loop_length are loop_length > 4 == 1+min_length_hairpin_loop
        #and 2 < nb_bits_position since 2+min_length_hairpin_loop == 5 <= length
        circuit.append(incr_incorrect.control((nb_bits_position-2)+nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state("(")+n_to_ctrl_state(0,nb_bits_position-2)),[loop_length[i] for i in range(2,nb_bits_position)]+[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #set loop_length to its normal value
        #in particular it makes easy to check that a loop_length is small enough for the strain_contribution to apply to certain multiloop (there <***>)
        circuit.append(decr_position,[loop_length[i] for i in range(nb_bits_position)])
        
        
        # 3.4) Compute and add energy of loop to energy
        
        add_loop_energy(*basic_args_to_call_append,k_left)
        
        
        # 3.4) Restore parameter qbits
        
        inversed(characterise_loop)(*basic_args_to_call_append,k_left)
        
    for k_left in range(length-1-min_length_hairpin_loop,length):
        # 3.2 again) Check ( matching
        circuit.append(incr_incorrect.control(nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
    
    #OPTIMISATION
    #use one less bit for folding[length-min_length_hairpin_loop<=k_left<length] just by considering ./)
    #use one less bit for folding[0<=k_left<=min_length_hairpin_loop] just by considering ./(
    
    
    # 4) Check <-.-> choice
    
    #|.->. and .<-.| are prohibited
    for k_dot in range(1,length-1):
        #if 0 == folding[k_dot][0] it is a .
        #then
        # - if 1 == folding[k_dot][1] it is a .->
        #   then
        #   (a) 1 == folding[k_dot-1][0] it is a |
        #   and
        #   (b) 0 == folding[k_dot+1][0] it is a .
        #   is prohibited
        #   so by flipping (a) and (b) if 1 == folding[k_dot][1]
        #   then one get
        #   (flipped a) 0 == folding[k_dot-1][0]
        #   and
        #   (flipped b) 1 == folding[k_dot+1][0]
        #   is prohibited
        #   that is the same condition as the other case since
        #   (flipped a) == (c)
        #   and
        #   (flipped b) == (d)
        #   
        # - if 0 == folding[k_dot][1] it is a <-.
        #   then
        #   (c) 0 == folding[k_dot-1][0] it is a .
        #   and
        #   (d) 1 == folding[k_dot+1][0] it is a |
        #   is prohibited
        
        #flip
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
        
        #test
        circuit.append(incr_incorrect.control(3*(nb_bits_bracket_dot-1),ctrl_state=bracket_dot_to_ctrl_state("|")+bracket_dot_to_ctrl_state(".")+bracket_dot_to_ctrl_state(".")),[folding[k_dot-1][0]]+[folding[k_dot][0]]+[folding[k_dot+1][0]]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #flip back
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
    
    
    # 5) substract the energy of the specification_folding
    
    #TODO: using loop_specification (which should probably be modified a bit)
    
    for k_left in range(length-1-min_length_hairpin_loop):
        if "(" == folding_specification[k_left]:
            k_right = loop_specification[k_left][0][3]
            loop_length_specification = loop_specification[k_left][0][3]
            
            #TODO
    
    
    return "RNA_design"


length = 5
folding_specification="(...)"*(length//5)+"."*(length % 5)

circ,_,_=to_circuit_builder(RNA_design)(folding_specification)

circ.draw()

┌───┐                                                       »
       (/./h/)0_0: ┤ H ├────────────■────■─────────────────────────────────────»
                   ├───┤            │    │                                     »
       (/./h/)0_1: ┤ H ├────────────┼────o─────────────────────────────────────»
                   ├───┤            │    │                                     »
       (/./h/)1_0: ┤ H ├────────────┼────┼─────────────────────■─────────■─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)1_1: ┤ H ├────────────┼────┼─────────────────────o─────────■─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)2_0: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)2_1: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)3_0: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)3_1: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)4_0: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       (/./h/)4_1: ┤ H ├────────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤            │    │                     │         │     »
       A/U/G/C0_0: ┤ H ├──o─────────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │         │    │                     │         │     »
       A/U/G/C0_1: ┤ H ├──┼────o────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C1_0: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C1_1: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C2_0: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C2_1: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C3_0: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   ├───┤  │    │    │    │                     │         │     »
       A/U/G/C3_1: ┤ H ├──┼────┼────┼────┼─────────────────────┼─────────┼─────»
                   └───┘┌─┴─┐  │    │    │                     │         │     »
       A/U/G/C4_0: ─────┤ X ├──┼────┼────┼─────────────────────┼─────────┼─────»
                        └───┘┌─┴─┐  │    │                     │         │     »
       A/U/G/C4_1: ──────────┤ X ├──┼────┼─────────────────────┼─────────┼─────»
                   ┌───┐     └───┘┌─┴─┐  │  ┌────────┐     ┌───┴────┐┌───┴────┐»
          count_0: ┤ X ├──■───────┤ X ├──┼──┤0       ├──o──┤0       ├┤0       ├»
                   └───┘  │       └───┘┌─┴─┐│        │  │  │        ││        │»
          count_1: ───────┼────────────┤ X ├┤1       ├──o──┤1       ├┤1       ├»
                          │            └───┘│  incr4 │  │  │  incr4 ││  decr4 │»
          count_2: ───────┼─────────────────┤2       ├──o──┤2       ├┤2       ├»
                          │                 │        │  │  │        ││        │»
          count_3: ───────┼─────────────────┤3       ├──o──┤3       ├┤3       ├»
                        ┌─┴─┐               └───┬────┘┌─┴─┐└───┬────┘└───┬────┘»
   continue_match: ─────┤ X ├───────────────────o─

statistics

In [268]:
print("width {} ; depth {}".format(circ.width(),circ.depth()))

width 56 ; depth 281


### 3.2 - Tests and simulations

In [269]:
np.set_printoptions(threshold=np.inf)

def semantics_identity(x):
    return x

def simulate_circuit(circuit,semantics=semantics_identity):
    backend = Aer.get_backend('statevector_simulator')
    nb_bits = circuit.width()
    job = execute(circuit, backend=backend, shots=1, memory=True)
    job_result = job.result()
    res=np.asarray(job_result.get_statevector(circuit))
    #state = np.asarray([(semantics(n_to_ctrl_state(i,nb_bits)),n_to_ctrl_state(i,nb_bits),val) for i,val in enumerate(res) if val>10**(-15)])
    state = np.asarray([(semantics(n_to_ctrl_state(i,nb_bits)),'   ') for i,val in enumerate(res) if val>10**(-12)])
    return circuit,state,res

def simulate(f,length):
    folding_specification="(...)"*(length//5)+"."*(length % 5)
    circ,sem,_=to_circuit_builder(f)(folding_specification)
    return simulate_circuit(circ,sem)



# Transpile for simulator
#simulator = Aer.get_backend('statevector_simulator')
#circ = transpile(circ, simulator)

# Run and get counts
#result = simulator.run(circ).result()
#counts = result.get_counts(circ)
#plot_histogram(counts, title='counts')

`initialise_folding()` works.

In [270]:
length = 6

_,state,_ = simulate(initialise_folding,length)

for (s,_) in state:
    print(s)


<<<<<<

(<<<<<

><<<<<

)<<<<<

<(<<<<

((<<<<

>(<<<<

)(<<<<

<><<<<

(><<<<

>><<<<

)><<<<

<)<<<<

()<<<<

>)<<<<

))<<<<

<<(<<<

(<(<<<

><(<<<

)<(<<<

<((<<<

(((<<<

>((<<<

)((<<<

<>(<<<

(>(<<<

>>(<<<

)>(<<<

<)(<<<

()(<<<

>)(<<<

))(<<<

<<><<<

(<><<<

><><<<

)<><<<

<(><<<

((><<<

>(><<<

)(><<<

<>><<<

(>><<<

>>><<<

)>><<<

<)><<<

()><<<

>)><<<

))><<<

<<)<<<

(<)<<<

><)<<<

)<)<<<

<()<<<

(()<<<

>()<<<

)()<<<

<>)<<<

(>)<<<

>>)<<<

)>)<<<

<))<<<

())<<<

>))<<<

)))<<<

<<<(<<

(<<(<<

><<(<<

)<<(<<

<(<(<<

((<(<<

>(<(<<

)(<(<<

<><(<<

(><(<<

>><(<<

)><(<<

<)<(<<

()<(<<

>)<(<<

))<(<<

<<((<<

(<((<<

><((<<

)<((<<

<(((<<

((((<<

>(((<<

)(((<<

<>((<<

(>((<<

>>((<<

)>((<<

<)((<<

()((<<

>)((<<

))((<<

<<>(<<

(<>(<<

><>(<<

)<>(<<

<(>(<<

((>(<<

>(>(<<

)(>(<<

<>>(<<

(>>(<<

>>>(<<

)>>(<<

<)>(<<

()>(<<

>)>(<<

))>(<<

<<)(<<

(<)(<<

><)(<<

)<)(<<

<()(<<

(()(<<

>()(<<

)()(<<

<>)(<<

(>)(<<

>>)(<<

)>)(<<

<))(<<


`initialise_sequence()` works.

In [271]:
length = 6

_,state,_ = simulate(initialise_sequence,length)

for (s,_) in state:
    print(s)


UAAAAA

UCAAAA

UGAAAA

UUAAAA

UACAAA

UCCAAA

UGCAAA

UUCAAA

UAGAAA

UCGAAA

UGGAAA

UUGAAA

UAUAAA

UCUAAA

UGUAAA

UUUAAA

UAACAA

UCACAA

UGACAA

UUACAA

UACCAA

UCCCAA

UGCCAA

UUCCAA

UAGCAA

UCGCAA

UGGCAA

UUGCAA

UAUCAA

UCUCAA

UGUCAA

UUUCAA

UAAGAA

UCAGAA

UGAGAA

UUAGAA

UACGAA

UCCGAA

UGCGAA

UUCGAA

UAGGAA

UCGGAA

UGGGAA

UUGGAA

UAUGAA

UCUGAA

UGUGAA

UUUGAA

UAAUAA

UCAUAA

UGAUAA

UUAUAA

UACUAA

UCCUAA

UGCUAA

UUCUAA

UAGUAA

UCGUAA

UGGUAA

UUGUAA

UAUUAA

UCUUAA

UGUUAA

UUUUAA

GAAACA

GCAACA

GGAACA

GUAACA

GACACA

GCCACA

GGCACA

GUCACA

GAGACA

GCGACA

GGGACA

GUGACA

GAUACA

GCUACA

GGUACA

GUUACA

GAACCA

GCACCA

GGACCA

GUACCA

GACCCA

GCCCCA

GGCCCA

GUCCCA

GAGCCA

GCGCCA

GGGCCA

GUGCCA

GAUCCA

GCUCCA

GGUCCA

GUUCCA

GAAGCA

GCAGCA

GGAGCA

GUAGCA

GACGCA

GCCGCA

GGCGCA

GUCGCA

GAGGCA

GCGGCA

GGGGCA

GUGGCA

GAUGCA

GCUGCA

GGUGCA

GUUGCA

GAAUCA

GCAUCA

GGAUCA

GUAUCA

GACUCA

GCCUCA

GGCUCA

GUCUCA

GAGUCA

GCGUCA

GGGUCA

GUGUCA

GAUUCA


`characterise_folding(compute_energy=false)` (that is checking the parenthesis matching in `folding`) works.

In [272]:
#this function is dedicated to tests
def check_folding(circuit,
               folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,use_right_matching_left_prev,use_right_matching_left_next,incorrect_configuration,energy,
               all_registers,
               nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
               all_nb_bits,
               incr_position,decr_position,add_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
               all_gates,
               length,sequence_specification,folding_specification,loop_specification,
               all_specifiers,
               basic_args_to_call_append):
    
    # 1) Initiation
    
    initialise_folding(*basic_args_to_call_append)
    
    # 2) Check ) matching
    
    #check if there is a folding[k_right] == ) non matched with a folding[k_left] == (
    #that would then match with a ( at an imaginary position folding[k_left=-1]
    #the result is in continue_searching_right_matching_left:
    # 1 == continue_searching_right_matching_left if ( == folding[k_left=-1] has not been matched
    # 0 == continue_searching_right_matching_left otherwise
    
    characterise_loop(*basic_args_to_call_append,-1,False,False)
    
    will_use_registers(circuit,[incorrect_configuration])
    
    #increment incorrect_configuration if necessary
    #(it is necessary to increment since the value of incorrect_configuration is unknown since characterise_loop(uncompute=False) can have changed it)
    circuit.append(incr_incorrect.control(ctrl_state="0"),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])

    inversed(characterise_loop)(*basic_args_to_call_append,-1,False,True)
    
    
    # 3) Check ( matching and add energ of loop
    
    for k_left in range(length-1-min_length_hairpin_loop):
        
        
        # 3.1) Compute parameters
        
        characterise_loop(*basic_args_to_call_append,k_left,False,False)
        
        
        # 3.2) Check ( matching
        
        #increment incorrect_configuration if folding[k_left] == ( has no matching )
        circuit.append(incr_incorrect.control(),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        
        # 3.3) Restore parameter qbits
        
        inversed(characterise_loop)(*basic_args_to_call_append,k_left,False,True)
        
    for k_left in range(length-1-min_length_hairpin_loop,length):
        # 3.2 again) Check ( matching
        circuit.append(incr_incorrect.control(nb_bits_bracket_dot,ctrl_state=bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
    
    
    # 4) Check <-.-> choice
    
    #|.->. and .<-.| are prohibited
    for k_dot in range(1,length-1):
        #if 0 == folding[k_dot][0] it is a .
        #then
        # - if 1 == folding[k_dot][1] it is a .->
        #   then
        #   (a) 1 == folding[k_dot-1][0] it is a |
        #   and
        #   (b) 0 == folding[k_dot+1][0] it is a .
        #   is prohibited
        #   so by flipping (a) and (b) if 1 == folding[k_dot][1]
        #   then one get
        #   (flipped a) 0 == folding[k_dot-1][0]
        #   and
        #   (flipped b) 1 == folding[k_dot+1][0]
        #   is prohibited
        #   that is the same condition as the other case since
        #   (flipped a) == (c)
        #   and
        #   (flipped b) == (d)
        #   
        # - if 0 == folding[k_dot][1] it is a <-.
        #   then
        #   (c) 0 == folding[k_dot-1][0] it is a .
        #   and
        #   (d) 1 == folding[k_dot+1][0] it is a |
        #   is prohibited
        
        #flip
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
        
        #test
        circuit.append(incr_incorrect.control(3*(nb_bits_bracket_dot-1),ctrl_state=bracket_dot_to_ctrl_state("|")+bracket_dot_to_ctrl_state(".")+bracket_dot_to_ctrl_state(".")),[folding[k_dot-1][0]]+[folding[k_dot][0]]+[folding[k_dot+1][0]]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #flip back
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
    
    
    return "check folding"


length = 5

_,state,_ = simulate(check_folding,length)

for (s,_) in state:
    print(s)
    print("\n")

0_w      0_s 0_c    
<<<<<


0_w      0_s 0_c    
><<<<


0_w      0_s 0_c    
<><<<


0_w      0_s 0_c    
>><<<


0_w      0_s 0_c    
()<<<


0_w      0_s 0_c    
<<><<


0_w      0_s 0_c    
><><<


0_w      0_s 0_c    
<>><<


0_w      0_s 0_c    
>>><<


0_w      0_s 0_c    
(<)<<


0_w      0_s 0_c    
(>)<<


0_w      0_s 0_c    
<<<><


0_w      0_s 0_c    
><<><


0_w      0_s 0_c    
<><><


0_w      0_s 0_c    
>><><


0_w      0_s 0_c    
()<><


0_w      0_s 0_c    
<<>><


0_w      0_s 0_c    
><>><


0_w      0_s 0_c    
<>>><


0_w      0_s 0_c    
>>>><


0_w      0_s 0_c    
(<>)<


0_w      0_s 0_c    
<<<<>


0_w      0_s 0_c    
><<<>


0_w      0_s 0_c    
<><<>


0_w      0_s 0_c    
>><<>


0_w      0_s 0_c    
()<<>


0_w      0_s 0_c    
<<><>


0_w      0_s 0_c    
><><>


0_w      0_s 0_c    
<>><>


0_w      0_s 0_c    
>>><>


0_w      0_s 0_c    
(<)<>


0_w      0_s 0_c    
(>)<>


0_w      0_s 0_c    
<<<>>


0_w      0_s 0_c    
><<>>


0_w      0_s 0

## 4 - Grover search

In [220]:
#amplify the sequences verifying:
#forall folding:
# - 0 != incorrect_configuration
# - or 0 <= energy
#TODO

# Draft

In [262]:
r = QuantumRegister(3)
circ = QuantumCircuit(r)

circ.x(r[0])
circ.cx(r[0],r[2])

q = QuantumRegister(3)
circuit = QuantumCircuit(q)

circuit.x(q[0])
circuit.cx(q[0],q[2])

circ.compose(circ,[r[i] for i in range(3)],inplace=True)
circ.draw()

#_,state,_ = simulate_circuit(circ)
#print(state)

┌───┐     ┌───┐     
q104610_0: ┤ X ├──■──┤ X ├──■──
           └───┘  │  └───┘  │  
q104610_1: ───────┼─────────┼──
                ┌─┴─┐     ┌─┴─┐
q104610_2: ─────┤ X ├─────┤ X ├
                └───┘     └───┘

In [ ]:
#def grey_code_to_n(n_grey_code):
#    return int("".join(reversed(["0" if bit else "1" for bit in n_grey_code])),2)
#
##to be used this way
#
#k_grey_code = [True]*nb_bits_position
#next_k_grey_code = [True]*nb_bits_position
#
#parity = True
#
#for _ in range((2**nb_bits_position)-1):#-1 beacause we avoid 0...0 == next_k_grey_code there [*]
#    #this part is only about dealing with k
#    #what is done once k gets its next value starts there [*]
#
#    #compute what would be the next_k_grey_code 
#    if parity:
#        next_k_grey_code[0] = not next_k_grey_code[0]
#    else:
#        first_one = 0
#        while nb_bits_position > first_one and next_k_grey_code[first_one]:
#            first_one+=1
#        #when 0...0 == next_k_grey_code
#        #then True == parity
#        #so nb_bits_position == first_one should never happend
#        if nb_bits_position-1 == first_one:
#            next_k_grey_code[nb_bits_position-1] = not next_k_grey_code[nb_bits_position-1]
#        else:
#            next_k_grey_code[first_one+1] = not next_k_grey_code[first_one+1]
#    parity = not parity
#
#    k = grey_code_to_n(next_k_grey_code)-1
#    
#    print(next_k_grey_code,k)
#    
#    if condition(k):
#        for i in range(nb_bits_position):
#            if k_grey_code[i] != next_k_grey_code[i]:
#                circuit.x(k_register[i])
#                k_grey_code[i] = next_k_grey_code[i]
#
#        #[*] k_register == k
